In [5]:
#Same as V3 , but updated so it reports Subset numbers
#gtv00_listPx is not used now

In [1]:
from os.path import join
from datetime import time, datetime
import os
import glob
import numpy as np
from skimage.draw import polygon
import SimpleITK as sitk
#import pydicom as dicom
#from pydicom.tag import Tag
import sys
import csv
import pandas as pd
import shutil

In [2]:
import yaml
with open('config_CheckNiiOk.yaml', 'r') as stream:
    config = yaml.safe_load(stream)
    
    itvtot = config['itv_tot_labels']
    itvtum = config['itv_tumor_labels']
    itvnod = config['itv_ln_labels']

    
    gtvtot = config['gtv_tot_labels']
    gtvtum = config['gtv_tumor_labels']
    gtvnod = config['gtv_ln_labels']
    gtv00 = config['gtv_expi00_labels']

    rootp_Nii = config['root_Nii']

    aveTags = config['aveTags']
    bpTags =  config['bPTags']

    savecsv_path = config['save_csv']
    fieldnames = config['fieldnames']

PxList = os.listdir(rootp_Nii)
print(len(PxList),'-',PxList)
target_labels = itvtot + itvtum+itvnod+gtvtot+gtvtum+gtvnod

892 - ['4025031', '2156167', '2096249', '2154195', '3932820', '7236265', '619086', '3007430', '147625', '7786608', '8728753', '2328910', '1217167', '8967750', '2131592', '7676177', '810589', '2894544', '9765730', '7513114', '4664141', '766115', '3307795', '9588899', '23840', '2324157', '2123820', '5443347', '4023673', '2856145', '5196470', '5317512', '5635761', '8716256', '368708', '3741290', '1034109', '5666561', '1353431', '5497325', '14504', '170672', '6465961', '2628217', '2788596', '6841834', '7233645', '2370962', '367466', '234628', '5686032', '8036407', '2186015', '9593888', '5889191', '403161', '9222012', '8168463', '32628', '3853415', '3464709', '2084312', '2758081', '9936840', '9193538', '561128', '9918855', '2277478', '5960268', '3558287', '654137', '13194', '2326539', '5468590', '3059294', '9212857', '4572900', '110867', '1136033', '7838986', '4965499', '2049190', '2196145', '2168497', '1149114', '2475708', '7626090', '5539952', '3157977', '7176049', '2334091', '3409329', '

In [3]:
gtv00

['GTVtumor_insp']

In [4]:
subsetA_count =0
subsetB_count =0
subsetC_count =0
missingGTV = 0
missingITV=0
missingPlanCT =0
missing2BP =0
missing10BP =0 
corruptedPx_dict = {}
gtv00_listPx = []
missingDelineation =0


with open(savecsv_path, mode='w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    for patientID in PxList:
        itvtot_b,itvtum_b,itvnod_b=(False,False,False)
        gtvtot_b,gtvtum_b,gtvnod_b=(False,False,False)
        BP0_b,BP10_b,BP20_b,BP30_b,BP40_b,BP50_b,BP60_b,BP70_b,BP80_b,BP90_b,BP100_b=(False,False,False,False,False,False,False,False,False,False,False)
        planCT_b = False
        check_b = False

        for root, dirs, files in os.walk(os.path.join(rootp_Nii,patientID), topdown=True):
            for f in files: 
                #Average
                if any(planct_tt in f.lower() for planct_tt in aveTags):
                    planCT_b = True
                #ITV
                if  any(itvtot_tt in f for itvtot_tt in itvtot):
                    itvtot_b = True
                if any(itvtum_tt in f for itvtum_tt in itvtum):
                    itvtum_b = True
                if any(itvnod_tt in f for itvnod_tt in itvnod):
                    itvnod_b = True
                #GTV
                if any(gtvtot_tt in f for gtvtot_tt in gtvtot):
                    gtvtot_b = True
                if any(gtvtum_tt in f for gtvtum_tt in gtvtum):
                    gtvtum_b = True
                if any(gtvnod_tt in f for gtvnod_tt in gtvnod):
                    gtvnod_b = True
                if any(gtv00_tt in f for gtv00_tt in gtv00):
                    gtv00_listPx.append(patientID)
                #BP
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["0%"]):
                    BP0_b =True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["10%"]):
                    BP10_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["20%"]):
                    BP20_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["30%"]):
                    BP30_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["40%"]):
                    BP40_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["50%"]):
                    BP50_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["60%"]):
                    BP60_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["70%"]):
                    BP70_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["80%"]):
                    BP80_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["90%"]):
                    BP90_b = True
                if any(bpTags_tt in f.lower() for bpTags_tt in bpTags["100%"]):
                    BP100_b = True
    
        if planCT_b and (itvtot_b or itvtum_b or itvnod_b) and (gtvtot_b or gtvtum_b or gtvnod_b) and (BP50_b or BP60_b) and (BP0_b,BP100_b,BP90_b):
            check_b = True
        #Subset SUM    
        
        if planCT_b and (itvtot_b or itvtum_b or itvnod_b):
            subsetA_count+=1
        if (BP0_b,BP100_b,BP90_b) and (BP50_b or BP60_b)  and (BP10_b and BP20_b) and (BP30_b and BP40_b) and (BP70_b and BP80_b) and (itvtot_b or itvtum_b or itvnod_b):
            subsetC_count+=1
        elif (BP50_b or BP60_b) and (gtvtot_b or gtvtum_b or gtvnod_b):
            subsetB_count+=1
            
        #ERRORS sum
        if not(itvtot_b or itvtum_b or itvnod_b):
            missingITV +=1
            corruptedPx_dict[patientID] = "MissingITV"
        if not(gtvtot_b or gtvtum_b or gtvnod_b):
            missingGTV +=1
            corruptedPx_dict[patientID] = "MissingGTV"
        if not((BP50_b or BP60_b) and (BP0_b,BP100_b,BP10_b,BP90_b)):
            missing2BP +=1
            corruptedPx_dict[patientID] = "Missing Peak Phases"
            #print(files)
        if not(planCT_b):
            missingPlanCT +=1
            corruptedPx_dict[patientID] = "Missing Plan CT"
        writer.writerow({
            "PatientID": patientID,"PlanCT": planCT_b,
            "ITV_Total": itvtot_b,"ITV_Tumor": itvtum_b,"ITV_Nodal": itvnod_b,
            "GTV_Total": gtvtot_b,"GTV_Tumor": gtvtum_b,"GTV_Nodal": gtvnod_b,
            "BP_0%": BP0_b,"BP_10%": BP10_b,"BP_90%": BP90_b,"BP_100%": BP100_b,
            "BP_20%": BP20_b,"BP_30%": BP30_b,
            "BP_40%": BP40_b,"BP_50%": BP50_b,"BP_60%": BP60_b,
            "BP_70%": BP70_b,"BP_80%": BP80_b,"Check":check_b
        })


['4_thorax_2mmc__20__i40f__3.nii.gz', 'GTV.nii.gz', 'GTVn_created.nii.gz', 'GTVtumor.nii.gz']
['2_thorax_2mm__standaard__b40f.nii.gz', 'GTV.nii.gz', 'GTVklieren.nii.gz', 'GTVtumor.nii.gz', 'ITV.nii.gz', 'ITVklieren.nii.gz', 'ITVtumor.nii.gz']
['5_4d_thorax_20__20__i30f__3__average_ct.nii.gz', 'GTV.nii.gz', 'GTVklieren.nii.gz', 'GTVtumor.nii.gz', 'ITV.nii.gz', 'ITVklieren.nii.gz', 'ITVtumor.nii.gz']
['5_4d_thorax_20__20__i30f__3__average_ct.nii.gz', 'GTV.nii.gz', 'GTVklieren.nii.gz', 'GTVtumor.nii.gz', 'ITV.nii.gz', 'ITVklieren.nii.gz', 'ITVtumor.nii.gz']
['2_thorax_2mm__standaard__b40f.nii.gz', 'GTV.nii.gz', 'GTVklieren.nii.gz', 'GTVtumor.nii.gz', 'ITV.nii.gz', 'ITVklieren.nii.gz', 'ITVtumor.nii.gz']
['2_thorax_2mm___b40f.nii.gz', 'GTV.nii.gz', 'GTVklieren.nii.gz', 'GTVtumor.nii.gz', 'ITV.nii.gz', 'ITVklieren.nii.gz', 'ITVtumor.nii.gz']
['2_thorax_2mm__standaard__b40f.nii.gz', 'GTV.nii.gz', 'ITV.nii.gz', 'ITVklieren.nii.gz', 'ITVtumor.nii.gz']
['2_thorax_2mm__standaard__b40f.nii.gz', '

['2_thorax_2mm__standaard__b40f.nii.gz', 'GTV.nii.gz', 'GTVtumor.nii.gz', 'ITV.nii.gz', 'ITVtumor.nii.gz']
['2_mm_thorax25mm.nii.gz', 'GTV.nii.gz', 'GTVklieren.nii.gz', 'GTVtumor.nii.gz', 'ITV.nii.gz', 'ITVklieren.nii.gz', 'ITVtumor.nii.gz']
['5_4dthorax_200_qr40_s3_average_ct__efov.nii.gz', 'GTV.nii.gz', 'IGTV.nii.gz']
[]
['5_4d_thorax_20__20__i30f__3__average_ct.nii.gz', 'GTV.nii.gz', 'GTVklieren.nii.gz', 'GTVtumor.nii.gz', 'ITV.nii.gz', 'ITVklieren.nii.gz', 'ITVtumor.nii.gz']
['2_thorax_2mm__standaard__b40f.nii.gz', 'GTV.nii.gz', 'GTVklieren.nii.gz', 'GTVtumor.nii.gz', 'ITVklieren.nii.gz', 'ITVtumor.nii.gz']
['2_thorax_2mm__standaard__b40f.nii.gz', 'GTV.nii.gz', 'GTVtumor.nii.gz', 'ITV.nii.gz', 'ITVtumor.nii.gz']
['10_thorax_2mm__standaard__b40f.nii.gz', 'GTV.nii.gz', 'GTVklieren.nii.gz', 'GTVtumor.nii.gz', 'ITV.nii.gz', 'ITVklieren.nii.gz', 'ITVtumor.nii.gz']
['3_thorax_2mm__standaard__b40f.nii.gz', 'GTV.nii.gz', 'GTVklieren.nii.gz', 'GTVtumor.nii.gz', 'ITV.nii.gz', 'ITVklieren.nii

['2_thorax_2mm___sabr__b40f.nii.gz', 'GTV.nii.gz', 'GTVklieren.nii.gz', 'GTVtumor.nii.gz', 'ITV.nii.gz', 'ITVklieren.nii.gz', 'ITVtumor.nii.gz']
['2_mm_thorax25mm.nii.gz', 'GTV.nii.gz', 'GTVklieren.nii.gz', 'GTVtumor.nii.gz', 'ITV.nii.gz', 'ITVklieren.nii.gz', 'ITVtumor.nii.gz']
['5_4d_thorax_20__20__i30f__3_g_average_ct.nii.gz', 'GTVklieren.nii.gz', 'GTVtumor.nii.gz', 'ITV.nii.gz', 'ITVklieren.nii.gz', 'ITVtumor.nii.gz']
['100_t0pr94_-_6ar23_-_50.nii.gz', '101_t10pr4_-_15ar32_-_69.nii.gz', '102_t20pr15_-_24ar60_-_122.nii.gz', '103_t30pr26_-_33ar72_-_120.nii.gz', '104_t40pr37_-_44ar77_-_110.nii.gz', '108_t80pr75_-_86ar46_-_103.nii.gz', '2_mm_thorax25mm.nii.gz', 'GTV.nii.gz', 'GTVklieren.nii.gz', 'GTVtumor.nii.gz', 'ITVtumor.nii.gz']
['2_thorax_2mm__standaard__b40f.nii.gz', 'GTVklieren.nii.gz', 'GTVtumor.nii.gz', 'ITV.nii.gz', 'ITVklieren.nii.gz', 'ITVtumor.nii.gz']
['2_thorax_2mm__standaard__b40f.nii.gz', 'GTV.nii.gz', 'GTVklieren.nii.gz', 'GTVtumor.nii.gz', 'ITV.nii.gz', 'ITVtumor.nii

In [5]:
print("A:",subsetA_count,"B: ",subsetB_count,"C:",subsetC_count)

A: 865 B:  289 C: 458


In [6]:
print("Missing Delineation",missingGTV,missingITV,"BP",missing2BP,"PlanCT",missingPlanCT)


Missing Delineation 26 24 BP 132 PlanCT 5


In [7]:
len(corruptedPx_dict)

169

In [8]:
corruptedPx_dict

{'2096249': 'MissingGTV',
 '7236265': 'Missing Peak Phases',
 '7513114': 'Missing Peak Phases',
 '766115': 'Missing Peak Phases',
 '23840': 'Missing Peak Phases',
 '4023673': 'Missing Peak Phases',
 '2856145': 'Missing Peak Phases',
 '5196470': 'Missing Peak Phases',
 '5317512': 'MissingGTV',
 '5635761': 'Missing Peak Phases',
 '8716256': 'MissingGTV',
 '3741290': 'Missing Peak Phases',
 '5666561': 'Missing Peak Phases',
 '367466': 'Missing Peak Phases',
 '8036407': 'Missing Peak Phases',
 '5889191': 'Missing Plan CT',
 '32628': 'Missing Peak Phases',
 '3853415': 'Missing Peak Phases',
 '2084312': 'MissingGTV',
 '2758081': 'Missing Peak Phases',
 '9193538': 'Missing Peak Phases',
 '9918855': 'MissingGTV',
 '654137': 'Missing Peak Phases',
 '13194': 'MissingGTV',
 '5468590': 'Missing Peak Phases',
 '1136033': 'MissingGTV',
 '4965499': 'MissingGTV',
 '2334091': 'MissingITV',
 '2370282': 'Missing Peak Phases',
 '9854449': 'Missing Peak Phases',
 '9646447': 'Missing Peak Phases',
 '246826'